# Low Rank Adaptation (LoRA) Training for FLUX.1-dev Model

This notebook trains a LoRA adapter specifically for the **FLUX.1-dev** model by Black Forest Labs.

## Key Differences from Stable Diffusion LoRA:

### Architecture Changes:
- **Base Model**: `black-forest-labs/FLUX.1-dev` (instead of Stable Diffusion 1.5)
- **Text Encoder**: T5EncoderModel (instead of CLIP)
- **Main Model**: FluxTransformer2DModel (instead of UNet2DConditionModel)
- **Scheduler**: FlowMatchEulerDiscreteScheduler (instead of DDPM)
- **Pipeline**: FluxPipeline (text2img, no img2img)

### Optimizations for RTX4090:
- **Resolution**: 1024x1024 (instead of 512x512)
- **Batch Size**: 1 (FLUX requires more VRAM)
- **Mixed Precision**: bf16 (optimal for FLUX)
- **LoRA Rank**: 16 (higher for better quality)
- **LoRA Alpha**: 32

### Generation Parameters:
- **Guidance Scale**: 3.5 (FLUX works better with lower guidance)
- **Steps**: 28 (optimal for FLUX)
- **Output**: Direct text2img generation

⚠️ **Requirements**: This notebook requires significant VRAM (8GB+ recommended) and the FLUX.1-dev model access on HuggingFace.

## Imports + Settings

In [1]:
# Install compatible versions - balance between too new and too old
%pip install pillow pillow-avif-plugin openai "diffusers==0.30.2" "huggingface_hub==0.24.6" accelerate peft datasets dotenv rich tqdm ipywidgets

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Configure logging

from rich.console import Console
from rich.logging import RichHandler
import logging

console = Console()

logging.basicConfig(
    level="INFO",
    format="%(message)s",
    datefmt="[%X]",
    handlers=[RichHandler(console=console)]
)
log = logging.getLogger("rich")

In [3]:
from pathlib import Path
import json, os, re, shutil

IMAGES_OLD_DIR = Path("./images-old")
IMAGES_NEW_DIR = Path("./images-new")

WORK_DIR = Path("./work")
CONVERTED_DIR = WORK_DIR / "converted"
CONVERTED_OLD = CONVERTED_DIR / "old"
CONVERTED_NEW = CONVERTED_DIR / "new"
LABELS_DIR = WORK_DIR / "labels"
DATASET_DIR = WORK_DIR / "dataset"
LORA_OUTPUT_DIR = WORK_DIR / "lora-weights"

for p in [WORK_DIR, CONVERTED_OLD, CONVERTED_NEW, LABELS_DIR, DATASET_DIR, LORA_OUTPUT_DIR]:
    p.mkdir(parents=True, exist_ok=True)

TARGET_EXT = ".jpg"

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "YOUR_OPENAI_API_KEY_HERE")
OPENAI_MODEL = "gpt-4o-mini"

LABEL_SCHEMA = [
    "perspective (e.g., top-down, eye-level, low-angle)",
    "background (describe setting, simplicity vs clutter)",
    "lighting (e.g., soft, harsh, high-key, low-key, golden-hour)",
    "color palette (e.g., muted, vibrant, pastel, monochrome)",
    "texture treatment (e.g., painterly, grainy, smooth)",
    "composition (e.g., centered subject, rule of thirds, negative space)",
    "mood (e.g., nostalgic, cheerful, moody, futuristic)",
]

BASE_MODEL_ID = "black-forest-labs/FLUX.1-dev"
USE_8BIT_ADAM = True
MIXED_PRECISION = "bf16"  # FLUX works better with bfloat16
SEED = 42

log.info("Settings loaded.")




[11:03:36] INFO     Settings loaded.                                                               ]8;id=33065;file:///tmp/ipykernel_30654/3235581259.py\3235581259.py]8;;\:]8;id=479914;file:///tmp/ipykernel_30654/3235581259.py#38\38]8;;\

## Convert images to JPGs

In [4]:
from PIL import Image
from tqdm.notebook import tqdm

def convert_folder(src: Path, dst: Path, target_ext: str = ".jpg"):
    count = 0
    for p in tqdm(sorted(src.rglob("*"))):
        if p.is_dir() or p.name.startswith("."):
            continue
        if p.suffix.lower() not in [".jpg", ".jpeg", ".png", ".avif", ".webp"]:
            # skip non-image files explicitly
            continue
        try:
            with Image.open(p) as im:
                im = im.convert("RGB")  # unify colorspace
                out_path = dst / (p.stem + target_ext)
                out_path.parent.mkdir(parents=True, exist_ok=True)
                if target_ext.lower() == ".jpg":                    
                    im.save(out_path, format="JPEG", quality=95, optimize=True)
                else:
                    im.save(out_path)
                count += 1
        except Exception as e:
            print(f"⚠️ Failed to convert {p}: {e}")
    return count

n_old = convert_folder(IMAGES_OLD_DIR, CONVERTED_OLD, TARGET_EXT)
n_new = convert_folder(IMAGES_NEW_DIR, CONVERTED_NEW, TARGET_EXT)
log.info(f"Converted {n_old} old images and {n_new} new images to {TARGET_EXT}.")



  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/31 [00:00<?, ?it/s]

[11:03:37] INFO     Converted 16 old images and 31 new images to .jpg.                             ]8;id=427505;file:///tmp/ipykernel_30654/1464447316.py\1464447316.py]8;;\:]8;id=852309;file:///tmp/ipykernel_30654/1464447316.py#28\28]8;;\

# Image labelling

In [5]:
from openai import AzureOpenAI
import os
import time
import base64

from dotenv import load_dotenv
load_dotenv()

AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_DEPLOYMENT_NAME = os.getenv("AZURE_DEPLOYMENT_NAME")
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")

client = AzureOpenAI(
    api_key=AZURE_OPENAI_API_KEY,
    api_version=AZURE_OPENAI_API_VERSION,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
)

LABEL_SCHEMA = [
    "food item (main dish, ingredient, or drink that appears)",
    "perspective (e.g., top-down, eye-level, low-angle)",
    "background (describe setting, simplicity vs clutter)",
    "lighting (e.g., soft, harsh, high-key, low-key, golden-hour)",
    "color palette (e.g., muted, vibrant, pastel, monochrome)",
    "texture treatment (e.g., painterly, grainy, smooth)",
    "composition (e.g., centered subject, rule of thirds, negative space)",
    "mood (e.g., nostalgic, cheerful, moody, futuristic)",
]


def b64_image(image_path: Path) -> str:
    with open(image_path, "rb") as f:
        return base64.b64encode(f.read()).decode("utf-8")

def load_existing_labels(jsonl_path: Path) -> set:
    """Load already labeled filenames from existing JSONL file"""
    labeled_files = set()
    if jsonl_path.exists():
        with open(jsonl_path, "r", encoding="utf-8") as f:
            for line in f:
                try:
                    data = json.loads(line.strip())
                    if "filename" in data:
                        labeled_files.add(str(Path(data["filename"]).resolve()))
                except json.JSONDecodeError:
                    continue
    return labeled_files

def label_image(img_path: Path, schema):
    img_b64 = b64_image(img_path)
    schema_bullets = "\n".join([f"- {s}" for s in schema])
    system_prompt = (
        "You are a meticulous food and style image annotator. "
        "Return a strict JSON object with fields exactly matching the requested schema."
    )
    user_prompt = f"""
Analyze the image and label it for the following attributes:
{schema_bullets}

Rules:
- Return strictly valid JSON (no markdown).
- For 'food item', use the most specific name possible (e.g., 'spaghetti carbonara', 'latte art', 'sushi roll').
- For style attributes, use short descriptive phrases.
- Include a 'notes' field for anything noteworthy.
"""
    try:
        resp = client.chat.completions.create(
            model=AZURE_DEPLOYMENT_NAME,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": [
                    {"type": "text", "text": user_prompt},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{img_b64}"}}
                ]},
            ],
            temperature=0.2,
        )
        text = resp.choices[0].message.content
        try:
            return json.loads(text)
        except Exception:
            m = re.search(r"\{[\s\S]*\}", text)
            log.error(f"⚠️ Non-JSON response for {img_path}:\n{text}")
            return json.loads(m.group(0)) if m else {"raw": text}
    except Exception as e:
        return {"error": str(e)}

def label_folder(src: Path, out_jsonl: Path, max_images: int = 10, override: bool = False):
    n = 0
    skipped = 0
    
    # Load existing labels if not overriding
    labeled_files = set() if override else load_existing_labels(out_jsonl)
    
    # Open file in append mode if not overriding and file exists
    mode = "w" if override or not out_jsonl.exists() else "a"
    
    with open(out_jsonl, mode, encoding="utf-8") as out:
        for p in tqdm(sorted(src.glob(f"*{TARGET_EXT}"))):
            if n >= max_images:
                print(f"⏹️ Stopping after {max_images} images")
                break
                
            # Check if already labeled
            if str(p.resolve()) in labeled_files:
                log.info(f"⏭️ Skipping {p.name} (already labeled)")
                skipped += 1
                continue
                
            log.info(f"Labeling {p}...")
            result = label_image(p, LABEL_SCHEMA)
            rec = {"filename": str(p.resolve()), "labels": result}
            out.write(json.dumps(rec, ensure_ascii=False) + "\n")
            n += 1
            time.sleep(0.2)            
    
    print(f"✅ Labeled {n} new images, skipped {skipped} already labeled → {out_jsonl}")

print("Labeling images via Azure OpenAI...")
label_folder(CONVERTED_NEW, LABELS_DIR / "new_labels.jsonl", max_images=50, override=False)
label_folder(CONVERTED_OLD, LABELS_DIR / "old_labels.jsonl", max_images=50, override=False)
print("🎉 Done.")

Labeling images via Azure OpenAI...


  0%|          | 0/31 [00:00<?, ?it/s]

[11:03:38] INFO     ⏭️ Skipping ananassalsa-met-kiwi-jalapeno-en-koriander.jpg (already labeled)    ]8;id=379177;file:///tmp/ipykernel_30654/412114492.py\412114492.py]8;;\:]8;id=516913;file:///tmp/ipykernel_30654/412114492.py#107\107]8;;\

           INFO     ⏭️ Skipping andijviestamp-met-gerookte-kip-en-zongedroogde-tomaat.jpg (already  ]8;id=177046;file:///tmp/ipykernel_30654/412114492.py\412114492.py]8;;\:]8;id=542366;file:///tmp/ipykernel_30654/412114492.py#107\107]8;;\
                    labeled)                                                                                       

           INFO     ⏭️ Skipping bananenbrood-choco-kokostopping-van-giorgias-kitchen.jpg (already   ]8;id=502047;file:///tmp/ipykernel_30654/412114492.py\412114492.py]8;;\:]8;id=416843;file:///tmp/ipykernel_30654/412114492.py#107\107]8;;\
                    labeled)                                                                                       

...

           INFO     ⏭️ Skipping pineapple-slush.jpg (already labeled)                               ]8;id=114688;file:///tmp/ipykernel_30654/412114492.py\412114492.py]8;;\:]8;id=613418;file:///tmp/ipykernel_30654/412114492.py#107\107]8;;\

           INFO     ⏭️ Skipping pitabroodjes-met-lamsvlees-en-tomatenrelish.jpg (already labeled)   ]8;id=667178;file:///tmp/ipykernel_30654/412114492.py\412114492.py]8;;\:]8;id=919326;file:///tmp/ipykernel_30654/412114492.py#107\107]8;;\

           INFO     ⏭️ Skipping pittige-vispakketjes.jpg (already labeled)                          ]8;id=308091;file:///tmp/ipykernel_30654/412114492.py\412114492.py]8;;\:]8;id=688580;file:///tmp/ipykernel_30654/412114492.py#107\107]8;;\

           INFO     ⏭️ Skipping roergebakken-spruitjes.jpg (already labeled)                        ]8;id=577181;file:///tmp/ipykernel_30654/412114492.py\412114492.py]8;;\:]8;id=246389;file:///tmp/ipykernel_30654/412114492.py#107\107]8;;\

✅ Labeled 0 new images, skipped 16 already labeled → work/labels/old_labels.jsonl
🎉 Done.


# Image captions

In [6]:
def captions_from_labels(jsonl_path: Path, out_dir: Path) -> int:
    """
    Reads a JSONL file with Azure OpenAI labeling results and builds
    caption .txt files next to each image for LoRA training.

    Captions include the food item first, followed by style attributes.
    Uses tqdm for progress and rich logging for clean notebook output.
    """
    out_dir.mkdir(parents=True, exist_ok=True)
    dst_img_dir = out_dir / "new"
    dst_img_dir.mkdir(parents=True, exist_ok=True)

    count = 0

    # Count lines first for tqdm total
    total = sum(1 for _ in open(jsonl_path, "r", encoding="utf-8"))

    log.info(f"📑 Building captions from {jsonl_path} ({total} records)...")

    with open(jsonl_path, "r", encoding="utf-8") as f:
        for line in tqdm(f, total=total, desc="Generating captions", leave=True):
            rec = json.loads(line)
            fp = Path(rec["filename"])
            labels = rec.get("labels", {})

            food = labels.get("food item") or labels.get("food item (main dish, ingredient, or drink that appears)")
            perspective = labels.get("perspective")
            background = labels.get("background")
            lighting = labels.get("lighting")
            palette = labels.get("color palette")
            texture = labels.get("texture treatment")
            composition = labels.get("composition")
            mood = labels.get("mood")

            parts = []
            if food:
                parts.append(f"a photo of {food}")
            if composition:
                parts.append(f"composition: {composition}")
            if perspective:
                parts.append(f"perspective: {perspective}")
            if lighting:
                parts.append(f"lighting: {lighting}")
            if palette:
                parts.append(f"color palette: {palette}")
            if texture:
                parts.append(f"texture: {texture}")
            if background:
                parts.append(f"background: {background}")
            if mood:
                parts.append(f"mood: {mood}")

            caption = ", ".join(parts) + "."
            if not caption.strip() or caption == ".":
                caption = "A food scene with distinctive style."

            out_img_path = dst_img_dir / fp.name
            out_txt_path = dst_img_dir / (fp.stem + ".txt")

            try:
                shutil.copy2(fp, out_img_path)
                with open(out_txt_path, "w", encoding="utf-8") as out:
                    out.write(caption)
                count += 1
            except Exception as e:
                log.error(f"⚠️ Failed to process {fp.name}: {e}")

    log.info(f"✅ Wrote captions for {count} images into {dst_img_dir}")
    return count

n1 = captions_from_labels(LABELS_DIR / "new_labels.jsonl", DATASET_DIR)
n2 = captions_from_labels(LABELS_DIR / "old_labels.jsonl", DATASET_DIR)
log.info(f"🎉 Done. Total captions created: {n1 + n2}")

           INFO     📑 Building captions from work/labels/new_labels.jsonl (31 records)...         ]8;id=187436;file:///tmp/ipykernel_30654/1215574807.py\1215574807.py]8;;\:]8;id=337272;file:///tmp/ipykernel_30654/1215574807.py#18\18]8;;\

Generating captions:   0%|          | 0/31 [00:00<?, ?it/s]

           ERROR    ⚠️ Failed to process ananassalsa-met-kiwi-jalapeno-en-koriander.jpg: [Errno 1]  ]8;id=545271;file:///tmp/ipykernel_30654/1215574807.py\1215574807.py]8;;\:]8;id=990670;file:///tmp/ipykernel_30654/1215574807.py#66\66]8;;\
                    Operation not permitted                                                                        

           ERROR    ⚠️ Failed to process andijviestamp-met-gerookte-kip-en-zongedroogde-tomaat.jpg: ]8;id=547367;file:///tmp/ipykernel_30654/1215574807.py\1215574807.py]8;;\:]8;id=69155;file:///tmp/ipykernel_30654/1215574807.py#66\66]8;;\
                    [Errno 1] Operation not permitted                                                              

           ERROR    ⚠️ Failed to process bananenbrood-choco-kokostopping-van-giorgias-kitchen.jpg:  ]8;id=754573;file:///tmp/ipykernel_30654/1215574807.py\1215574807.py]8;;\:]8;id=405947;file:///tmp/ipykernel_30654/1215574807.py#66\66]8;;\
                    [Errno 1] Operation not permitted                                                              

...

           ERROR    ⚠️ Failed to process pitabroodjes-met-lamsvlees-en-tomatenrelish.jpg: [Errno 1] ]8;id=950660;file:///tmp/ipykernel_30654/1215574807.py\1215574807.py]8;;\:]8;id=172214;file:///tmp/ipykernel_30654/1215574807.py#66\66]8;;\
                    Operation not permitted                                                                        

           ERROR    ⚠️ Failed to process pittige-vispakketjes.jpg: [Errno 1] Operation not          ]8;id=508553;file:///tmp/ipykernel_30654/1215574807.py\1215574807.py]8;;\:]8;id=69742;file:///tmp/ipykernel_30654/1215574807.py#66\66]8;;\
                    permitted                                                                                      

           ERROR    ⚠️ Failed to process roergebakken-spruitjes.jpg: [Errno 1] Operation not        ]8;id=842692;file:///tmp/ipykernel_30654/1215574807.py\1215574807.py]8;;\:]8;id=672305;file:///tmp/ipykernel_30654/1215574807.py#66\66]8;;\
                    permitted                                                                                      

           INFO     ✅ Wrote captions for 0 images into work/dataset/new                           ]8;id=939735;file:///tmp/ipykernel_30654/1215574807.py\1215574807.py]8;;\:]8;id=688150;file:///tmp/ipykernel_30654/1215574807.py#68\68]8;;\

           INFO     🎉 Done. Total captions created: 0                                             ]8;id=865458;file:///tmp/ipykernel_30654/1215574807.py\1215574807.py]8;;\:]8;id=477787;file:///tmp/ipykernel_30654/1215574807.py#73\73]8;;\

# Model training

In [7]:
# Check PyTorch version and fix if needed
import torch
print(f"Current PyTorch version: {torch.__version__}")

# Check if we have a proper PyTorch installation
try:
    # Test basic PyTorch functionality
    x = torch.tensor([1.0, 2.0])
    print(f"PyTorch tensor test: {x}")
    print(f"CUDA available: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"CUDA device count: {torch.cuda.device_count()}")
        print(f"Current CUDA device: {torch.cuda.current_device()}")
except Exception as e:
    print(f"PyTorch test failed: {e}")

# The issue might be that the development version isn't recognized by transformers
# Let's patch the version check by setting a compatible version string
import sys
if hasattr(torch, '__version__'):
    original_version = torch.__version__
    if 'a0' in torch.__version__:
        print(f"Detected development PyTorch version: {original_version}")
        print("Patching version string for compatibility...")
        # Set a stable version string that transformers will accept
        torch.__version__ = "2.1.0"
        print(f"Patched PyTorch version to: {torch.__version__}")

Current PyTorch version: 2.1.0a0+b5021ba
PyTorch tensor test: tensor([1., 2.])
CUDA available: True
CUDA device count: 1
Current CUDA device: 0
Detected development PyTorch version: 2.1.0a0+b5021ba
Patching version string for compatibility...
Patched PyTorch version to: 2.1.0


In [8]:
import torch, random
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from transformers import T5EncoderModel, T5Tokenizer
from diffusers import FluxTransformer2DModel, AutoencoderKL, FlowMatchEulerDiscreteScheduler
from accelerate import Accelerator
from peft import LoraConfig, get_peft_model

random.seed(SEED)
torch.manual_seed(SEED)

TRAIN_BATCH_SIZE = 1  # FLUX requires more memory, reduce batch size
LR = 1e-4
MAX_STEPS = 5000
IMG_RES = 1024  # FLUX works at higher resolution
LORA_R = 16  # Higher rank for better quality
LORA_ALPHA = 32
LORA_DROPOUT = 0.1

log.info("Loading models...")
tokenizer = T5Tokenizer.from_pretrained(BASE_MODEL_ID, subfolder="tokenizer")
text_encoder = T5EncoderModel.from_pretrained(BASE_MODEL_ID, subfolder="text_encoder")
vae = AutoencoderKL.from_pretrained(BASE_MODEL_ID, subfolder="vae")
transformer = FluxTransformer2DModel.from_pretrained(BASE_MODEL_ID, subfolder="transformer")

# LoRA configuration for FLUX transformer
# Using FEATURE_EXTRACTION because we want to fine-tune the transformer attention layers
# while keeping text encoder and VAE frozen to preserve base language understanding
lora_config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    target_modules=["to_q", "to_k", "to_v", "to_out.0"],  # Transformer attention modules
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type="FEATURE_EXTRACTION",  # Correct for transformer-based models
)
transformer = get_peft_model(transformer, lora_config)
log.info("LoRA applied to Transformer")

class CaptionImageDataset(Dataset):
    def __init__(self, img_dir: Path, tokenizer, size=1024):
        self.paths = sorted((img_dir).glob("*.jpg"))
        self.tokenizer = tokenizer
        self.size = size
        log.info(f"Dataset created with {len(self.paths)} images")
    def __len__(self): return len(self.paths)
    def __getitem__(self, idx):
        img_path = self.paths[idx]
        txt_path = img_path.with_suffix(".txt")
        with open(txt_path, "r", encoding="utf-8") as f:
            caption = f.read().strip()
        image = Image.open(img_path).convert("RGB").resize((self.size, self.size), Image.BICUBIC)
        import numpy as np
        arr = np.array(image).astype("float32")/255.0
        arr = arr.transpose(2,0,1)
        arr = 2.0 * arr - 1.0
        image_tensor = torch.from_numpy(arr)
        ids = self.tokenizer(caption, truncation=True, padding="max_length",
            max_length=self.tokenizer.model_max_length, return_tensors="pt").input_ids[0]
        return {"pixel_values": image_tensor, "input_ids": ids}

train_ds = CaptionImageDataset(DATASET_DIR / "new", tokenizer, IMG_RES)
train_loader = DataLoader(train_ds, batch_size=TRAIN_BATCH_SIZE, shuffle=True, drop_last=True)

noise_scheduler = FlowMatchEulerDiscreteScheduler.from_pretrained(BASE_MODEL_ID, subfolder="scheduler")
accelerator = Accelerator(mixed_precision=MIXED_PRECISION if MIXED_PRECISION in ("fp16","bf16") else "no")
device = accelerator.device
log.info(f"Using device: {device}")

vae.to(device); text_encoder.to(device); transformer.to(device)

try:
    if USE_8BIT_ADAM:
        from bitsandbytes.optim import AdamW8bit
        optimizer = AdamW8bit((p for p in transformer.parameters() if p.requires_grad), lr=LR)
        log.info("Using 8-bit AdamW optimizer")
    else:
        raise Exception("use torch adamw")
except Exception:
    from torch.optim import AdamW
    optimizer = AdamW((p for p in transformer.parameters() if p.requires_grad), lr=LR)
    log.info("Using standard AdamW optimizer")

transformer, optimizer, train_loader = accelerator.prepare(transformer, optimizer, train_loader)
global_step = 0
transformer.train()

log.info("Starting training...")
for epoch in range(999999):
    for batch in train_loader:
        if global_step >= MAX_STEPS:
            break
        with accelerator.accumulate(transformer):
            pixel_values = batch["pixel_values"].to(device)
            input_ids = batch["input_ids"].to(device)
            latents = vae.encode(pixel_values).latent_dist.sample() * vae.config.scaling_factor
            noise = torch.randn_like(latents)
            timesteps = torch.randint(0, noise_scheduler.config.num_train_timesteps, (latents.shape[0],), device=device).long()
            noisy_latents = noise_scheduler.add_noise(latents, noise, timesteps)
            encoder_hidden_states = text_encoder(input_ids)[0]
            model_pred = transformer(
                hidden_states=noisy_latents,
                timestep=timesteps,
                encoder_hidden_states=encoder_hidden_states,
            ).sample
            loss = torch.nn.functional.mse_loss(model_pred, noise, reduction="mean")
            accelerator.backward(loss)
            optimizer.step(); optimizer.zero_grad()
        if accelerator.is_main_process and global_step % 50 == 0:
            log.info(f"step {global_step} | loss {loss.item():.4f}")
        global_step += 1
    if global_step >= MAX_STEPS: break

if accelerator.is_main_process:
    transformer.save_pretrained(str(LORA_OUTPUT_DIR))
    log.info(f"Saved LoRA weights to {LORA_OUTPUT_DIR}")

/usr/local/lib/python3.10/dist-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/v2/__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https:/

[11:03:44] INFO     Loading models...                                                              ]8;id=234053;file:///tmp/ipykernel_30654/2619882089.py\2619882089.py]8;;\:]8;id=146316;file:///tmp/ipykernel_30654/2619882089.py#20\20]8;;\

[11:03:46] INFO     LoRA applied to UNet                                                           ]8;id=571858;file:///tmp/ipykernel_30654/2619882089.py\2619882089.py]8;;\:]8;id=91161;file:///tmp/ipykernel_30654/2619882089.py#38\38]8;;\

           INFO     Dataset created with 47 images                                                 ]8;id=229258;file:///tmp/ipykernel_30654/2619882089.py\2619882089.py]8;;\:]8;id=243962;file:///tmp/ipykernel_30654/2619882089.py#45\45]8;;\

[11:03:47] INFO     Using device: cuda                                                             ]8;id=750800;file:///tmp/ipykernel_30654/2619882089.py\2619882089.py]8;;\:]8;id=681453;file:///tmp/ipykernel_30654/2619882089.py#68\68]8;;\

...

[11:30:16] INFO     step 4800 | loss 0.0122                                                       ]8;id=976031;file:///tmp/ipykernel_30654/2619882089.py\2619882089.py]8;;\:]8;id=431712;file:///tmp/ipykernel_30654/2619882089.py#106\106]8;;\

[11:30:31] INFO     step 4850 | loss 0.1305                                                       ]8;id=850132;file:///tmp/ipykernel_30654/2619882089.py\2619882089.py]8;;\:]8;id=260222;file:///tmp/ipykernel_30654/2619882089.py#106\106]8;;\

[11:30:49] INFO     step 4900 | loss 0.0779                                                       ]8;id=113346;file:///tmp/ipykernel_30654/2619882089.py\2619882089.py]8;;\:]8;id=401124;file:///tmp/ipykernel_30654/2619882089.py#106\106]8;;\

[11:31:05] INFO     step 4950 | loss 0.0807                                                       ]8;id=233238;file:///tmp/ipykernel_30654/2619882089.py\2619882089.py]8;;\:]8;id=209267;file:///tmp/ipykernel_30654/2619882089.py#106\106]8;;\

[11:31:21] INFO     Saved LoRA weights to work/lora-weights                                       ]8;id=860394;file:///tmp/ipykernel_30654/2619882089.py\2619882089.py]8;;\:]8;id=833980;file:///tmp/ipykernel_30654/2619882089.py#112\112]8;;\

# Image generation

In [ ]:
from diffusers import FluxPipeline
from peft import PeftModel
import torch


pipe = FluxPipeline.from_pretrained(
    BASE_MODEL_ID, torch_dtype=torch.bfloat16 if MIXED_PRECISION=="bf16" else torch.float32
).to(accelerator.device)
pipe.transformer = PeftModel.from_pretrained(pipe.transformer, LORA_OUTPUT_DIR)

CANONICAL_STYLE_PROMPT = (
    "food photography, professional culinary styling, top-down composition, "
    "minimal background, soft natural lighting, vibrant colors, "
    "high quality, detailed textures, appetizing presentation"
)

OUT_DIR = WORK_DIR / "outputs"
OUT_DIR.mkdir(parents=True, exist_ok=True)


guidance_scale = 3.5  # FLUX works better with lower guidance
num_inference_steps = 28  # FLUX optimal steps

height = 1024
width = 1024

# Generate a few sample images using FLUX with trained LoRA
sample_prompts = [
    "delicious pasta dish with herbs and parmesan cheese",
    "fresh salad with colorful vegetables and dressing", 
    "grilled chicken with roasted vegetables",
    "chocolate dessert with berries and cream"
]

for i, base_prompt in enumerate(sample_prompts):
    full_prompt = f"{base_prompt}, {CANONICAL_STYLE_PROMPT}"
    result = pipe(
        prompt=full_prompt,
        guidance_scale=guidance_scale,
        num_inference_steps=num_inference_steps,
        height=height,
        width=width,
    ).images[0]
    out_path = OUT_DIR / f"flux_generated_{i:02d}.jpg"
    result.save(out_path, "JPEG", quality=95, optimize=True)
    print(f"✅ Generated {out_path.name} with prompt: {full_prompt}")


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]